In [1]:
%load_ext autoreload
%autoreload 2

import sentences, DCS
import json
import numpy as np

In [21]:
D = json.load(open('../NewData/data7.json'))

In [25]:
print(len(D.keys()))
d_keys = list(D.keys())
print(d_keys[0])

276
120


In [23]:
for k in d_keys:
    ki = abs(int(k))
    if(ki >= 190 and ki <= 240):
        print(k)

-200
-230
-220
-240
-190
-210


In [45]:
def _getPathValue(terminal_1, path_type, terminal_2):
    p = 0;
    return p
        
def GetPathVector(terminal_1, terminal_2):
    x = np.array([0]*len(all_path_types))
    for i in range(len(all_path_types)):
        path = all_path_types[i]
        x[i] = _getPathValue(terminal_1, path, terminal_2)        
    return x

In [46]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Counting CNG GROUPS

In [10]:
import pickle
from sentences import *
from DCS import *
import json
import numpy as np
from romtoslp import *
from collections import defaultdict

In [2]:
loaded_DCS = pickle.load(open('../Simultaneous_DCS.p', 'rb'))

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT.p', 'rb'))
len(loaded_SKT)

100078

In [58]:
def cng_Groups(sentenceObj, dDict):
    #print('SKT ANALYZE')
    #print('-'*15)
    #print(sentenceObj.sentence)
    zz = 0
    # (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain) = SentencePreprocess(sentenceObj)
    # for cid in chunkDict.keys():
    #     print('Analyzing:', rom_slp(sentenceObj.chunk[cid].chunk_name))
    #     for pos in chunkDict[cid].keys():
    #         tupIds = chunkDict[cid][pos]
    #         for ti in tupIds:
    #             print('%d :' % (pos, ), end = ' ')
    #             print(tuplesMain[ti][0][1], end=' ')
    #             for tup in tuplesMain[ti]:
    #                 print([zz, tup[2], tup[3]], end=' ')
    #                 zz += 1
    #             print('')
    #     print('-'*25)

    for chunk in sentenceObj.chunk:
        # print("Analyzing ", rom_slp(chunk.chunk_name))
        for pos in chunk.chunk_words.keys():
            for word_sense in chunk.chunk_words[pos]:
                #word_sense = fix_w_new(word_sense)
                #print(word_sense.forms)
                for form_dict in word_sense.forms:
                    if isinstance(form_dict, dict):                        
                        for key, val_arr in form_dict.items():
                            #print('Key: ', key)
                            #print('Val Array: ', val_arr)
                            for val in val_arr:
                                if isinstance(val, list):
                                    if len(val) > 1:
                                        print('Key: ', key)
                                        print('Bad Val: ', val, ' Size: ', len(val))
                                    dDict[key].add(val[0])
                                else:
                                    dDict[key].add(val)
                # for formsDict in word_sense.forms:
                #     print(getCNGs(formsDict))

In [61]:
DD = defaultdict(lambda:set())
i = 0
for key, val in loaded_SKT.items():
    i += 1
    cng_Groups(val, DD)
    if i==1000:
        break

In [63]:
pickle.dump(dict(DD), open('cng_grps_dd.p', 'wb'), protocol=2)

# # Verb CNG Groups

In [1]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json

### Lem2CNG and CNG2Lem Matrices

In [2]:
mat_lem2cng = pickle.load(open('../NewData/mat_lem2cng.p', 'rb'), encoding='utf-8')
mat_cng2lem = pickle.load(open('../NewData/mat_cng2lem.p', 'rb'), encoding='utf-8')

In [13]:
all_cngs = list(int(vc) for vc in list(mat_cng2lem.keys()))
print(all_cngs[0:10])
print(len(all_cngs))

[71, -35, 116, 175, -123, 28, 36, -83, -52, 99]
276


In [4]:
tense_classes = defaultdict(lambda:set())
for vc in all_cngs:
    if vc < 0:
        tense_classes[str(-math.floor(vc/10))].add((-vc)%10)
tense_classes

defaultdict(<function __main__.<lambda>>,
            {'1': {0},
             '10': {1, 2, 3, 4, 6, 7, 8, 9},
             '11': {1, 2, 3, 4, 9},
             '12': {1, 2, 3, 4, 5, 7, 9},
             '13': {1, 2, 3, 6, 9},
             '14': {1, 2, 3, 7, 9},
             '15': {0, 1, 2, 3, 4, 7, 9},
             '16': {1, 2, 3, 4, 6, 7, 8, 9},
             '17': {1, 3, 6, 9},
             '19': {0},
             '2': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '20': {0},
             '21': {0},
             '22': {0},
             '23': {0},
             '24': {0},
             '25': {1, 2, 3, 5, 6, 7, 9},
             '26': {0},
             '27': {1, 2, 3, 6, 8, 9},
             '28': {1, 3, 6, 9},
             '29': {3},
             '3': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '30': {1, 2, 3, 6, 7, 9},
             '31': {1, 2, 3, 6, 7, 8, 9},
             '4': {1, 2, 3, 4, 5, 6, 7, 8, 9},
             '5': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
             '6': {1, 2, 3, 4, 5, 6,

In [8]:
# num_person: encoding style
verb_groups = defaultdict(lambda: set())
number_forms = dict()
number_forms['sg'] = [1, 2, 3]
number_forms['du'] = [4, 5, 6]
number_forms['pl'] = [7, 8, 9]

person_forms = dict()
person_forms['fp'] = [1, 4, 7]
person_forms['sp'] = [2, 5, 8]
person_forms['tp'] = [3, 6, 9]

comb_forms = dict()
comb_forms2 = dict()
for key_num, num_vals in number_forms.items():
    for key_person, person_vals in person_forms.items():
        s = set()
        for n in num_vals:
            for p in person_vals:
                if n==p:
                    s.add(n)
        comb_forms2['%s_%s'%(key_num, key_person)] = s
comb_forms.update(number_forms)
comb_forms.update(person_forms)
comb_forms2.update(number_forms)
comb_forms2.update(person_forms)
display(comb_forms)
display(comb_forms2)

{'du': [4, 5, 6],
 'fp': [1, 4, 7],
 'pl': [7, 8, 9],
 'sg': [1, 2, 3],
 'sp': [2, 5, 8],
 'tp': [3, 6, 9]}

{'du': [4, 5, 6],
 'du_fp': {4},
 'du_sp': {5},
 'du_tp': {6},
 'fp': [1, 4, 7],
 'pl': [7, 8, 9],
 'pl_fp': {7},
 'pl_sp': {8},
 'pl_tp': {9},
 'sg': [1, 2, 3],
 'sg_fp': {1},
 'sg_sp': {2},
 'sg_tp': {3},
 'sp': [2, 5, 8],
 'tp': [3, 6, 9]}

In [11]:
verb_groups = dict()
for tense in tense_classes.keys():
    for form, mods in comb_forms.items():
        new_key = '%s_%s'%(tense, form)
        s = set()
        for modulo in mods:
            if modulo in tense_classes[tense]:
                s.add(int(tense)*10 + modulo)
        if len(s) > 0:
            verb_groups[new_key] = list(s)

for form, mods in comb_forms2.items():
    new_key = form
    s = set()
    for modulo in mods:
        for tense in tense_classes.keys():
            if modulo in tense_classes[tense]:
                s.add(int(tense)*10 + modulo)
    if len(s) > 0:
        verb_groups[new_key] = list(s)
json.dump(verb_groups, open('json_n_pickle_Files/verbgroups.json', 'w'))
# verb_groups

In [12]:
len(verb_groups)

140

# # Noun CNG Groups

In [1]:
import pickle
from sentences import *
from DCS import *
from IPython.display import display
import json

In [3]:
noungroups = json.load(open('json_n_pickle_Files/noungroups.json'))

In [5]:
len(noungroups)

54

# # PMI

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
from sentences import *
from DCS import *
from collections import defaultdict
import math
from IPython.display import display
import json
from ProbModels import *

In [3]:
# IMPORT THE DATALOADER FILES FIRST
from utilities import *
from collections import *
import pandas as pd
import numpy as np
print("Dataloader Started[Prob]...")

fullCo_oc_mat = pickleFixLoad('extras/all_dcs_lemmas_matrix_countonly.p')
unigram_counts = pickleFixLoad('../NewData/ultimate_new_lastsem/unigram_lemma_countonly.p')

cng2cngFullMat = np.mat(pickle.load(open('extras/all_dcs_cngs_matrix_countonly.p','rb'), encoding = 'latin1'))
cng2index_dict = pickleFixLoad('cng2index_dict.p')

w2w_samecng_fullmat = pickle.load(open('extras/lemmas_with_same_cngs_matrix_countonly.p', 'rb'), encoding=u'utf-8')
samecng_unigram_counts = pickle.load(open('extras/dictionary_for_lemmas_with_same_cng.p', 'rb'), encoding=u'utf-8')

v2c_fullMat = pickle.load(open('extras/verbs_vs_cngs_matrix_countonly.p', 'rb'), encoding=u'utf-8')

print("Dataloader Finished[Prob]...")

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
pbModel = ProbModels(fullCo_oc_mat = fullCo_oc_mat, unigram_counts = unigram_counts,
               cng2cngFullMat = cng2cngFullMat, cng2index_dict = cng2index_dict,
               w2w_samecng_fullmat=w2w_samecng_fullmat, samecng_unigram_counts = samecng_unigram_counts,
               v2c_fullMat = v2c_fullMat)

In [5]:
lemma_list = list(unigram_counts.keys())
print(len(lemma_list))
print(lemma_list[0], lemma_list[10], lemma_list[100])
print(pbModel.kn_word2word(lemma_list[0], lemma_list[10]))

66914
avarRaka advAra pratisrotas
2.2026413966350114e-06


In [6]:
total_co_oc_count = pbModel.total_co_oc

In [7]:
# pmi(x;y) = -log2[p(x|y)/p(x)]
def PMI(x, y, _pbModel):
    if(x in _pbModel.fullCo_oc_mat[y]):
        p_x_given_y = _pbModel.fullCo_oc_mat[y][x]/_pbModel.unigram_counts[y]
        p_x = _pbModel.unigram_counts[x]/_pbModel.unigram_total_count
        #print(p_x_given_y, ' ', p_x)
        return -math.log2(p_x_given_y/p_x)
    else:
        p_x_given_y = 1/_pbModel.unigram_counts[y]
        p_x = _pbModel.unigram_counts[x]/_pbModel.unigram_total_count
        #print(p_x_given_y, ' ', p_x)
        return -math.log2(p_x_given_y/p_x)

In [8]:
print(PMI('vEdiSa', 'SatruGAtin', pbModel))
print(PMI('SatruGAtin', 'vEdiSa', pbModel))

print(PMI('anupavyAyacCamAna', 'SatruGAtin', pbModel))
print(PMI('SatruGAtin', 'anupavyAyacCamAna', pbModel))

-19.307367596880127
-19.307367596880127
-20.629295691767492
-20.629295691767492


In [ ]:
full_PMI_Mat = defaultdict(lambda: {})
count = 0
for y, _ in fullCo_oc_mat.items():
    count += 1
    for x, _ in fullCo_oc_mat[y].items():
        # print(y, x, fullCo_oc_mat[y][x])
        # Getting PMI(x | y)
        try:
            v = PMI(x, y, pbModel)
        except:
            v = None
        if v != None:
            full_PMI_Mat[y][x] = v
    if(count % 10000 ==0):
        print("Checkpoint: ", count)
full_PMI_Mat = dict(full_PMI_Mat)
pickle.dump(full_PMI_Mat, open('../NewData/ultimate_new_lastsem/full_PMI_Mat.p', 'wb'))

In [ ]:
print(y, x)
print(pbModel.fullCo_oc_mat[y][x]/pbModel.unigram_counts[y])
print(pbModel.unigram_counts[x])

In [ ]:
from utilities import *
import pickle

In [3]:
fullCo_oc_mat = pickleFixLoad('extras/all_dcs_lemmas_matrix_countonly.p')

In [4]:
list(fullCo_oc_mat.keys())[0:3]

['strImalApahA', 'anaSitum', 'vijihma']